<a href="https://colab.research.google.com/github/trappertwo/rPPG_experiments/blob/main/compress_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
### UBFC-rPPG dataset in drive folder
### (TODO) Figure out how to import data set directly into colab
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
### Compress the UBFC dataset and store the compressed files under a separate
### dir structure. Only needs to be run once.
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import shutil
import os

!pip install -q mediapy
import mediapy as media

base_path = '/content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_2' # @param {type:"string"}
compressed_path = '/content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_40_1_train' # @param {type:"string"}

!ls -l $base_path/subject9

total 1814623
-r-------- 1 root root      96774 Mar  7  2018 ground_truth.txt
-r-------- 1 root root 1858075976 Nov  2  2017 vid.avi


In [12]:
# Compress data - creates new dataset
COMPRESSION_RATIO = 40 # @param {type:"number"}
LENGTH = 640 # @param {type:"number"}
WIDTH = 480 # @param {type:"number"}
FPS = 30 # @param {type:"number"}
ORIGINAL_BIT_RATE = LENGTH*WIDTH*8*3*FPS #Only for UBFC-rPPG
print("Original bit rate: " + str(ORIGINAL_BIT_RATE))
COMPRESSION_BIT_RATE = ORIGINAL_BIT_RATE // COMPRESSION_RATIO
print("Compression bit rate: " + str(COMPRESSION_BIT_RATE))

dirnames = os.listdir(base_path)
if not os.path.isdir(compressed_path):
  os.mkdir(compressed_path)
for dirname in dirnames:
  if not os.path.isdir(os.path.join(compressed_path, dirname)):
    os.mkdir(os.path.join(compressed_path, dirname))
  for filename in os.listdir(os.path.join(base_path, dirname)):
    if filename.endswith('.avi'):
      video = media.read_video(os.path.join(base_path, dirname, filename))
      compressed_data = media.compress_video(video, bps=COMPRESSION_BIT_RATE, fps=30)
      print("Writing " + os.path.join(compressed_path, dirname, 'vid.mp4'))
      with open(os.path.join(compressed_path, dirname, 'vid.mp4'), 'wb') as f:
        f.write(compressed_data)
    else:
      shutil.copy(os.path.join(base_path, dirname, filename), os.path.join(compressed_path, dirname, filename))
print("All files compressed")

Original bit rate: 221184000
Compression bit rate: 11059200
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_20_1_train/subject1/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_20_1_train/subject10/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_20_1_train/subject11/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_20_1_train/subject12/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_20_1_train/subject13/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_20_1_train/subject14/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_20_1_train/subject15/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_20_1_train/subject16/vid.mp4
Writing /content/drive/MyDrive/research_project/data/UBFC-rPPG/DATASET_20_1_train/subject17/vid.mp4
Writing /content/drive/MyDrive/research_p

In [15]:
# Compare file sizes:
original_vid = media.read_video(base_path+'/subject1/vid.avi')
new_vid = media.read_video(compressed_path+'/subject1/vid.mp4')
print(original_vid.size / new_vid.size)

1.0
